In [52]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np   
import pandas as pd  

import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [53]:
RAW_DATA_PATH = '/Users/piyush/Desktop/dsml_Portfolio/visa/data/raw/EasyVisa.csv'
df = pd.read_csv(RAW_DATA_PATH)
df.drop(['case_id'],axis=1,inplace=True)

In [54]:
df.head()

,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,case_status
0,Asia,High School,N,N,14513,2007,West,592.2029,Hour,Y,Denied
1,Asia,Master's,Y,N,2412,2002,Northeast,83425.6500,Year,Y,Certified
2,Asia,Bachelor's,N,Y,44444,2008,West,122996.8600,Year,Y,Denied
3,Asia,Bachelor's,N,N,98,1897,West,83434.0300,Year,Y,Denied
4,Africa,Master's,Y,N,1082,2005,South,149907.3900,Year,Y,Certified


In [55]:
df.loc[df["unit_of_wage"]=="Hour", "prevailing_wage"] = df.loc[df["unit_of_wage"]=="Hour", "prevailing_wage"]*52

In [56]:
df["unit_of_wage"] = df["unit_of_wage"].astype('object')
df.loc[df["unit_of_wage"]=="Hour", "unit_of_wage"] = "Hourly"
df.loc[df["unit_of_wage"]=="Week", "unit_of_wage"] = "Not_Hourly"
df.loc[df["unit_of_wage"]=="Month", "unit_of_wage"] = "Not_Hourly"
df.loc[df["unit_of_wage"]=="Year", "unit_of_wage"] = "Not_Hourly"
df["unit_of_wage"] = df["unit_of_wage"].astype('category')

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25480 entries, 0 to 25479
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   continent              25480 non-null  object  
 1   education_of_employee  25480 non-null  object  
 2   has_job_experience     25480 non-null  object  
 3   requires_job_training  25480 non-null  object  
 4   no_of_employees        25480 non-null  int64   
 5   yr_of_estab            25480 non-null  int64   
 6   region_of_employment   25480 non-null  object  
 7   prevailing_wage        25480 non-null  float64 
 8   unit_of_wage           25480 non-null  category
 9   full_time_position     25480 non-null  object  
 10  case_status            25480 non-null  object  
dtypes: category(1), float64(1), int64(2), object(7)
memory usage: 2.0+ MB


In [58]:
df["no_of_employees"] = pd.cut( df["no_of_employees"], \
                                  [0, 2500, 7500, np.inf], \
                                  labels = ["Small_Sized(<2500employees)", \
                                            "Medium_Sized(2500-7500employees)", \
                                            "Large_Sized(>75000employees)"])
#df["no_of_employees"] = df["no_of_employees"].astype("category")

In [59]:
df

,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,case_status
0,Asia,High School,N,N,Large_Sized(>75000employees),2007,West,30794.5508,Hourly,Y,Denied
1,Asia,Master's,Y,N,Small_Sized(<2500employees),2002,Northeast,83425.6500,Not_Hourly,Y,Certified
2,Asia,Bachelor's,N,Y,Large_Sized(>75000employees),2008,West,122996.8600,Not_Hourly,Y,Denied
3,Asia,Bachelor's,N,N,Small_Sized(<2500employees),1897,West,83434.0300,Not_Hourly,Y,Denied
4,Africa,Master's,Y,N,Small_Sized(<2500employees),2005,South,149907.3900,Not_Hourly,Y,Certified
...,...,...,...,...,...,...,...,...,...,...,...
25475,Asia,Bachelor's,Y,Y,Medium_Sized(2500-7500employees),2008,South,77092.5700,Not_Hourly,Y,Certified
25476,Asia,High School,Y,N,Medium_Sized(2500-7500employees),2006,Northeast,279174.7900,Not_Hourly,Y,Certified
25477,Asia,Master's,Y,N,Small_Sized(<2500employees),1910,South,146298.8500,Not_Hourly,N,Certified
25478,Asia,Master's,Y,Y,Small_Sized(<2500employees),1887,West,86154.7700,Not_Hourly,Y,Certified


In [60]:
df["yr_of_estab"] = pd.cut( df["yr_of_estab"], \
                                  [-np.inf, 1990, np.inf], \
                                  labels = ["Older_Estab(1800-1990)", \
                                            "Newer_Estab(>1990)"])
df["yr_of_estab"] = df["yr_of_estab"].astype("category")

In [61]:
df

,continent,education_of_employee,has_job_experience,requires_job_training,no_of_employees,yr_of_estab,region_of_employment,prevailing_wage,unit_of_wage,full_time_position,case_status
0,Asia,High School,N,N,Large_Sized(>75000employees),Newer_Estab(>1990),West,30794.5508,Hourly,Y,Denied
1,Asia,Master's,Y,N,Small_Sized(<2500employees),Newer_Estab(>1990),Northeast,83425.6500,Not_Hourly,Y,Certified
2,Asia,Bachelor's,N,Y,Large_Sized(>75000employees),Newer_Estab(>1990),West,122996.8600,Not_Hourly,Y,Denied
3,Asia,Bachelor's,N,N,Small_Sized(<2500employees),Older_Estab(1800-1990),West,83434.0300,Not_Hourly,Y,Denied
4,Africa,Master's,Y,N,Small_Sized(<2500employees),Newer_Estab(>1990),South,149907.3900,Not_Hourly,Y,Certified
...,...,...,...,...,...,...,...,...,...,...,...
25475,Asia,Bachelor's,Y,Y,Medium_Sized(2500-7500employees),Newer_Estab(>1990),South,77092.5700,Not_Hourly,Y,Certified
25476,Asia,High School,Y,N,Medium_Sized(2500-7500employees),Newer_Estab(>1990),Northeast,279174.7900,Not_Hourly,Y,Certified
25477,Asia,Master's,Y,N,Small_Sized(<2500employees),Older_Estab(1800-1990),South,146298.8500,Not_Hourly,N,Certified
25478,Asia,Master's,Y,Y,Small_Sized(<2500employees),Older_Estab(1800-1990),West,86154.7700,Not_Hourly,Y,Certified


In [62]:
df["case_status"] = df["case_status"].apply(lambda x: 0 if x == "Denied" else 1)


In [63]:
df.case_status.value_counts()

1    17018
0     8462
Name: case_status, dtype: int64

In [64]:
X = df.drop("case_status", axis=1)
y = df["case_status"]
X = pd.get_dummies(X, drop_first=True)
#Splitting the data into train and test sets in 70:30 ratio
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1, stratify=y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1, stratify=y)

In [51]:
y.shape

(25480,)

In [66]:
X.to_csv('/Users/piyush/Desktop/dsml_Portfolio/visa/data/processed/X.csv',index=None)
y.to_csv('/Users/piyush/Desktop/dsml_Portfolio/visa/data/processed/y.csv',index=None)


In [15]:
X_train.shape, X_test.shape


((17836, 20), (7644, 20))

In [16]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [17]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6756933542647828


In [19]:
clf.score(X_test,y_test)

0.6756933542647828

In [20]:
cross_val_score(RandomForestClassifier(n_estimators=100),X,y,cv=5)

array([0.68229984, 0.67209576, 0.68838305, 0.63952119, 0.62264521])

In [67]:
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
    XGBClassifier()
]

In [68]:
# 5 Cross-validation
CV = 10
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, X, y, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [69]:
mean_accuracy = cv_df.groupby('model_name').accuracy.mean()
std_accuracy = cv_df.groupby('model_name').accuracy.std()

acc = pd.concat([mean_accuracy, std_accuracy], axis= 1, 
          ignore_index=True)
acc.columns = ['Mean Accuracy', 'Standard deviation']
acc

,Mean Accuracy,Standard deviation
model_name,,
LinearSVC,0.624137,0.107681
LogisticRegression,0.667896,0.000165
MultinomialNB,0.708870,0.066151
RandomForestClassifier,0.726962,0.057713
XGBClassifier,0.732300,0.055040
